<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #모두의노코드 게시글, 한국 버블 사용자 커뮤니티 오픈톡방 내용 전처리 코드(로컬)

In [ ]:
!pip install soynlp
import pandas as pd
import numpy as np
import re
import os
import tensorflow as tf
import urllib.request
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 3.1 MB/s eta 0:00:00


## 카카오톡 대화내용 전처리

In [ ]:
#카카오톡 텍스트 로드 후 정규표현식으로 메시지 구분
#C:\Users\벅성현\Desktop
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/KakaoTalkData0417.txt'
with open(file_path, 'r', encoding = 'utf-8') as f:
  file_content = f.read()
  # "https"를 포함하는 모든 링크 삭제(https 발견 시 그 다음에 나타나는 공백 바로 전까지의 문자열을 링크로 간주)
  file_content = re.sub(r'https?:\/\/\S+', '', file_content)

Mounted at /content/drive


In [ ]:
date_pattern = re.compile(r'\d{4}년 \d{1,2}월 \d{1,2}일')

def split_messages_by_date(text):
    # 날짜 위치를 찾아 리스트로 저장
    dates = [match.start() for match in date_pattern.finditer(text)]
    messages = []
    send_dates = []

    # 각 날짜 위치를 기준으로 메시지 분리
    for i in range(len(dates)):
        start = dates[i]
        # 마지막 날짜라면, 텍스트의 끝까지를 메시지로 추출
        end = dates[i + 1] if i + 1 < len(dates) else len(text)
        message = text[start:end].strip()
        idx_dot = message.find(':') #시간 사이에 있는 : 위치 찾아내기
        idx_comma, idx_dot = message.find(','), message.find(':', idx_dot + 1)
        if (idx_dot == -1): #누군가 메시지를 보낸 줄이 아닌 경우를 모두 포함, 이때는 메시지를 추가하지 않고 넘어간다.
          continue
        else:
          #메시지 보낸 사람, 메시지 내용, 보낸 날짜로 message 구성
          message = [message[idx_comma + 1 : idx_dot - 1], message[idx_dot + 1 :], message[0 : idx_comma]]
        messages.append(message)

    return messages

messages = split_messages_by_date(file_content)
print(messages[:5])

[[' 하희철 / 기획, 마케팅, 노코드', ' 안녕하세요~', '2022년 3월 18일 오후 1:25'], [' (알 수 없음)', ' 안녕하세요!', '2022년 3월 18일 오후 5:42'], [' 하희철 / 기획, 마케팅, 노코드', ' 방 만든지 몇분 만에 들어오셔서 놀랐습니다 ㅎㅎ', '2022년 3월 18일 오후 5:43'], [' 하희철 / 기획, 마케팅, 노코드', ' 노코드 방에서 오신건가요?', '2022년 3월 18일 오후 5:43'], [' (알 수 없음)', ' 아 안녕하세요', '2022년 3월 18일 오후 5:43']]


In [ ]:
#메시지에 유니코드에 해당하는 이모티콘이 포함되어 있으면 삭제하는 함수
def remove_emojis(text):
    # 이모티콘에 해당하는 유니코드 패턴
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # emoticons
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
# '@이름' 제거 함수
def remove_names(text):
  for name in names:
    text = re.sub(r'@' + re.escape(name[1:]), '', text) #df['name'] 열을 보면 각 행의 맨 처음에 공백이 하나 있음
  return text

In [ ]:
#첫 번째 요소는 사람 이름, 두 번째 요소는 텍스트 내용을 가지고 있는 리스트를 묶은 리스트를 pandas로 읽기
df = pd.DataFrame(messages, columns = ['name', 'text', 'date'])

#날짜 비교를 위해 날짜 형식 변경
#'date' 열을 datetime 객체로 변환
df['date'] = pd.to_datetime(df['date'], format='%Y년 %m월 %d일', exact = False)

#datetime 객체를 'YYYY-MM-DD' 형식의 문자열로 변환
df['date'] = df['date'].dt.strftime('%Y-%m-%d')

#메시지에 .png, jpg, 삭제된 메시지입니다, 사진 읽지 않음, 동영상 읽지 않음 포함하면 삭제
del_ids = df[df['text'].str.contains('\.png|\.jpg|삭제된 메시지입니다|사진 읽지 않음|동영상 읽지 않음')].index #df[df['text'].str.contains('<사진 읽지 않음>') | df['text'].str.contains('.png') | df['text'].str.contains('삭제된 메시지입니다') | df['text'].str.contains('<동영상 읽지 않음>')].index
df = df.drop(del_ids)

#메시지에 '사진'이나 '사진 2장'이나 '동영상'만 포함하는 행을 찾아 삭제
df = df[~df['text'].str.match(r'^(사진|사진 \d+장|동영상)$')]

#메시지에 유니코드에 해당하는 이모티콘이 포함되어 있으면 삭제
df['text'] = df['text'].apply(remove_emojis)

#메시지에 '@이름' 형식으로 태그한 내용이 있으면 삭제
names = set(df['name'].to_list())

# 데이터프레임 열에 함수 적용
df['text'] = df['text'].apply(remove_names)

#메시지 첫 글자가 [면 해당 메시지 삭제, 홍보글의 형식임.
df = df[~df['text'].str.startswith(' [')]

#똑같은 사람이 연속적으로 메시지를 보냈다면 메시지들을 하나의 행으로 통합해서 문맥정보 담기
# 'name' 열이 이전 행과 다른지 여부를 나타내는 불리언 시리즈 생성
name_changed = df['name'] != df['name'].shift(1)

# 'name_changed'의 누적합을 통해 'group' 열 생성
df['group'] = name_changed.cumsum()

# 'group'과 'name' 열을 기준으로 그룹화하고, 'text' 열의 값 합치기, 'date' 값은 그룹 내 마지막 메시지의 날짜로 설정
df = df.groupby(['group', 'name']).agg({'text': '\n'.join, 'date': 'max'}).reset_index()
#df = df.groupby(['group', 'name'])['text'].agg('\n'.join).reset_index()
df = df.drop(['group'], axis = 1)

#줄바꿈 문자(\n) 제거하기
pattern = r'\n'
df = df.replace(pattern, '', regex = True)

#결측치 제거. 확인해보니 한 행에 링크만 존재했던 경우 링크를 삭제하니 결측치(빈 문자열)가 되었다.
#빈 문자열을 nan으로 변환하고 제거
df = df.replace('', np.nan).dropna()

In [ ]:
text = df['text'].to_list()

#결측치 검증
print(type(text[:5]))
print(all(isinstance(item, str) for item in text))
for i, item in enumerate(text):
  if not isinstance(item, str):
    print(f"Index {i}: {item} (Type: {type(item)})")

<class 'list'>
True


In [ ]:
# for i, item in enumerate(df):
#   if not isinstance(item, str):
#     print(f"Index {i}: {item} (Type: {type(item)})")

In [ ]:
df

,name,text,date
0,"하희철 / 기획, 마케팅, 노코드",안녕하세요~,2022-03-18
1,(알 수 없음),안녕하세요!,2022-03-18
2,"하희철 / 기획, 마케팅, 노코드",방 만든지 몇분 만에 들어오셔서 놀랐습니다 ㅎㅎ 노코드 방에서 오신건가요?,2022-03-18
3,(알 수 없음),아 안녕하세요 아니요 검색해서 들어왔습니다,2022-03-18
4,"하희철 / 기획, 마케팅, 노코드",그렇군요~ 반갑습니다,2022-03-18
...,...,...,...
17222,이창호 / 에어빌드,"저도 궁금한데 어차피 schedule도 action에서 넣어서 실행되는 거니깐, ...",2024-04-17
17223,민호,앗뜨… schedule 데이터 테이블 시트를 따로 만들어야 하는군요..,2024-04-17
17224,이창호 / 에어빌드,이거는 지금 딱 생각난 방법이라서 다른 깔끔한 방법이 있을지도 모르겠습니다 ㅎㅎ,2024-04-17
17225,민호,저도 더 찾아보고 있다면 공유드리겠습니다! 답변 주셔서 감사합니다 :),2024-04-17


In [ ]:
print(len(df))

7859


In [ ]:
# 제어 문자 제거 함수
def remove_control_characters(s):
    return ''.join(char for char in s if ord(char) >= 32 or char in '\t\n\r')

# 문자열 열에 함수 적용
df['text'] = df['text'].apply(remove_control_characters)

In [ ]:
#.xlsx 파일로 google drive에 저장
talk_save_path = '/content/drive/My Drive/talk_preprocess_result_long.xlsx'

df.to_excel(talk_save_path)

## 모두의노코드 커뮤니티 게시글 전처리

In [ ]:
#커뮤니티 글 로드
qna_path = '/content/drive/My Drive/community_qna.xlsx'
all_contents_path = '/content/drive/My Drive/community_all_contents.csv'
all_comments_path = '/content/drive/My Drive/community_all_comments.csv'

df_qna = pd.read_excel(qna_path)

#qna 파일의 링크 형식 수정
df_qna['게시글 링크'] = 'https://everynocode.org/content/' + df_qna['Slug'].astype(str)

#_x1008_같은 기호를 자동으로 삭제하기 위해 cp949 인코딩으로 읽은 뒤 utf-8 인코딩으로 재처리
df_all_contents_cp949 = pd.read_csv(all_contents_path, encoding = 'cp949')
df_all_contents_cp949.to_csv('/content/drive/My Drive/community_all_contents_utf8.csv', encoding = 'utf-8')
df_all_contents = pd.read_csv('/content/drive/My Drive/community_all_contents_utf8.csv', encoding = 'utf-8')

df_all_comments_cp949 = pd.read_csv(all_comments_path, encoding = 'cp949')
df_all_comments_cp949.to_csv('/content/drive/My Drive/community_all_comments_utf8.csv', encoding = 'utf-8')
df_all_comments = pd.read_csv('/content/drive/My Drive/community_all_comments_utf8.csv', encoding = 'utf-8')

print(df_qna)
print(df_all_contents)
print(df_all_comments)

                                                    내용  \
0    안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.\n\n이렇게 첫번째 질문...   
1    저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...   
2    안녕하세요.\n모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. \...   
3    [color=rgb(6, 40, 61)]헤더그룹 안에 로고와 CTA를 넣으시고 ma...   
4    안녕하세요.\n제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.\n어제...   
..                                                 ...   
573  정보 구조나 와이어 프레임을 그릴 때 보통 피그마를 쓰시는 분이 많은데 한글로 된 ...   
574  [img width=100%]//3146248aee8f8e5c27832897c3a0...   
575  안녕하세요!\n \n현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하...   
576  안녕하세요! 버블로 외주개발하고 있는 리트머스팀입니다.\n\nyongjun@cigr...   
577  안녕하세요~\n\n아래 링크로 톡 남겨놓으시면 견적 도와드리겠습니다!\n\nhttp...   

                                                    답변  \
0    저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...   
1                                                  NaN   
2    [color=rgb(6, 40, 61)]헤더그룹 안에 로고와 CTA를 넣으시고 ma...   
3                                                  NaN   
4    충분히 버블로 

In [ ]:
#[img = ...]와 같은 태그 종류 구하기

# 괄호 안의 문자열을 찾아 집합에 추가하는 함수
def find_brackets(text):
    pattern = r'\[([^\]]+)\]'
    matches = re.findall(pattern, text)
    pattern_set = set(matches)
    return pattern_set
# 데이터프레임에 대해 함수 적용
string = ', '.join(df_qna['내용'].dropna(axis = 0).to_list())
pattern_set = find_brackets(string)
string = ', '.join(df_all_contents['내용'].dropna(axis = 0).to_list())
pattern_set = pattern_set.union(find_brackets(string))
string = ', '.join(df_all_comments['내용'].dropna(axis = 0).to_list())
pattern_set = pattern_set.union(find_brackets(string))

# 결과 출력
print("Found bracketed text:", pattern_set)

Found bracketed text: {'url=https://blog.naver.com/synctree/223304697132', '/h2', '/img', 'url=https://open.kakao.com/o/gG3iydke', 'li indent=2 align=left', '페이지소스 보기', 'highlight=rgba(2, 2, 2, 0.06)', 'indent data=3', 'url=https://open.kakao.com/o/sbPOMi3f', 'ul', '/s', '수정', 'settings', 'url=https://support.google.com/tagmanager/answer/7182738?hl=ko', 'url=https://bit.ly/44TJHJZ', 'url=https://ko-kr.facebook.com/business/help/777099232674791?id=1205376682832142', 'url=https://brunch.co.kr/@catchup/16', 'highlight=rgba(0, 0, 0, 0)', 'url=https://blog.naver.com/synctree/223251415924', 'url=https://www.swapstack.co/', '텍스트1 -이미지1', 'url=https://event-us.kr/m/62088/5424', '3', 'highlight=rgb(248, 248, 248)', '/highlight', 'url=https://prompti.kr', '0', 'url=https://www.behance.net/gallery/176891703/NHR-X-Litmers', 'url=https://syncathon-2.vercel.app', 'url=https://api.ncloud-docs.com/docs/common-ncpapi#%EC%9D%B8%EC%A6%9D%ED%82%A4%EC%83%9D%EC%84%B1%ED%95%98%EA%B8%B0', 'url=mailto:brandcan

In [ ]:
#데이터프레임 모든 셀에서 [ul], [ol] 등과 같은 태그 삭제, 이때 pattern_set에서 참조해서 직접 삭제([] 안에 유의미한 정보가 들어가있기도 하기 때문)
# + url 형식, 이미지 형식, 줄바꿈 텍스트 삭제
# + 이모티콘 인식 못해서 ??로 출력된 것들 다 삭제
#삭제할 패턴 : [url=https ...],[size=정수],[/size],[ol data=정수],[/ol],[index],[/h3],[b],[/url],[color=rgb(정수, 정수, 정수)],[/b],[/img],[highlight=rgb(정수, 정수, 정수)],[ol],[color=var(--tertiary)],[indent data=정수],[/indent],[/i],[/h4],[/ul],[li indent=정수 align=left 또는 right 또는 등등?],[/color],[h4],[/code],[h3],[/highlight],[ul],[ul data=정수],[highlight=rgba(정수, 정수, 정수, 정수)],[u],[/color],[center],[i],[color=inherit],[/li],[/ml],[code],[/youtube],[youtube],[img width=정수%],[/u],[color=var(--primary-medium)],[ml],[/center],[highlight=var(--primary-low)]

pattern = r'\n|\?\?|\S+\.jpeg|\S+\.png|https:\/\/[^\s]+|\[url=https[^\]]*\]|\[\/url\]|\[size=\d+\]|\[\/size\]|\[b\]|\[\/b\]|\[color=rgb\(\d+,\s*\d+,\s*\d+\)\]|\[\/color\]|\[color=[^\]]*\]|\[highlight=rgb\(\d+,\s*\d+,\s*\d+\)\]|\[\/highlight\]|\[highlight=rgba\(\d+,\s*\d+,\s*\d+,\s*\d+\)\]|\[ol data=\d+\]|\[\/ol\]|\[index\]|\[\/index\]|\[h3\]|\[\/h3\]|\[h4\]|\[\/h4\]|\[ul\]|\[ul data=\d+\]|\[\/ul\]|\[li[^\]]*\]|\[\/li\]|\[indent data=\d+\]|\[\/indent\]|\[ml\]|\[\/ml\]|\[youtube\]|\[\/youtube\]|\[u\]|\[\/u\]|\[center\]|\[\/center\]|\[i\]|\[\/i\]|\[highlight=[^\]]*\]|\[ol\]|\[font=".*?"\]|\[/font\]|\[/?h[12]\]|\[highlight=.*?\]|\[/?quote\]|\[/?s\]|\[img width=\d+%\].*?\[/img\]|\[code\].*?\[/code\]|\[img\]'

text_data = df_qna[['내용', '답변', '댓글']].replace(pattern, '', regex=True)
df_qna[['내용', '답변', '댓글']] = text_data

text_data = df_all_contents[['내용', '답변', '댓글']].replace(pattern, '', regex=True)
df_all_contents[['내용', '답변', '댓글']] = text_data

text_data = df_all_comments[['내용', '대댓글', '댓글']].replace(pattern, '', regex=True)
df_all_comments[['내용', '대댓글', '댓글']] = text_data

KeyError: 'date'

In [ ]:
#df_qna의 작성날짜를 카카오톡 데이터의 작성날짜 형식과 똑같도록 변경
df_qna['작성일'] = df_qna['작성일'].dt.strftime('%Y-%m-%d')

In [ ]:
print(df_qna)

                                                    내용  \
0    안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...   
1    저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...   
2    안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
3    헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요...   
4    안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...   
..                                                 ...   
573  정보 구조나 와이어 프레임을 그릴 때 보통 피그마를 쓰시는 분이 많은데 한글로 된 ...   
574                                                      
575  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
576  안녕하세요! 버블로 외주개발하고 있는 리트머스팀입니다.yongjun@cigro.io...   
577                 안녕하세요~아래 링크로 톡 남겨놓으시면 견적 도와드리겠습니다!   

                                                    답변  \
0    저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...   
1                                                  NaN   
2    헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요...   
3                                                  NaN   
4    충분히 버블로 

In [ ]:
# NaN 값을 공백('')으로 변환
df_qna.fillna('', inplace=True)
df_all_contents.fillna('', inplace=True)
df_all_comments.fillna('', inplace=True)

In [ ]:
#유니코드에 해당하는 이모티콘 제거
df_qna['내용'] = df_qna['내용'].apply(remove_emojis)
df_qna['답변'] = df_qna['답변'].apply(remove_emojis)
df_qna['댓글'] = df_qna['댓글'].apply(remove_emojis)

In [ ]:
df_all_contents['내용'] = df_all_contents['내용'].apply(remove_emojis)
df_all_contents['답변'] = df_all_contents['답변'].apply(remove_emojis)
df_all_contents['댓글'] = df_all_contents['댓글'].apply(remove_emojis)

In [ ]:
df_all_comments['내용'] = df_all_comments['내용'].apply(remove_emojis)
df_all_comments['대댓글'] = df_all_comments['대댓글'].apply(remove_emojis)
df_all_comments['댓글'] = df_all_comments['댓글'].apply(remove_emojis)

In [ ]:
# print(df_qna.iloc[4]['내용'])

In [ ]:
#전처리된 세 파일을 저장
df_qna.to_excel('/content/drive/My Drive/community_qna_preprocessed.xlsx')
df_all_contents.to_csv('/content/drive/My Drive/community_all_contents_preprocessed.csv')
df_all_comments.to_csv('/content/drive/My Drive/community_all_comments_preprocessed.csv')